# **Normalizzazione dei Nomi Aziendali**

## **Introduzione**

Nel processo di deduplicazione e record linkage, è essenziale normalizzare i nomi aziendali per evitare variazioni dovute a abbreviazioni, formati o errori di battitura. In questa fase, implementiamo una strategia per rendere i nomi uniformi e comparabili.

---

## **Step 1: Importazione delle Librerie e del DataFrame**

In [10]:
import re
import pandas as pd
import unidecode

AZIENDE_CSV = '../aziende.csv'
IMPIEGATI_CSV = '../impiegati.csv'
companies_df = pd.read_csv(AZIENDE_CSV)

C:\Users\emigi\AppData\Local\Temp\ipykernel_14944\3147525914.py:7: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)


---

## **Step 2: Definizione della Mappa di Sostituzione**
Utilizziamo un dizionario per standardizzare suffissi comuni nei nomi aziendali.
### **Questo va sicuramente migliorato magari passando tutta la prima colonna a chat**

In [11]:
REPLACEMENTS = {
    r"&": "",
    r"(?<=\s|,|\.)inc\.?\b": "",
    r"(?<=\s|,|\.)ltd\b": "",
    r"(?<=\s|,|\.)llc\b": "",
    r"(?<=\s|,|\.)plc\b": "",
    r"(?<=\s|,|\.)co\.?\b": "",  # Handles "co." and "co,"
    r"(?<=\s|,|\.)corp\.?\b": "",
    r"(?<=\s|,|\.)srl\b": "",
    r"(?<=\s|,|\.)spa\b": "",
    r"(?<=\s|,|\.)ag\b": "",
    r"(?<=\s|,|\.)sa\b": "",
    r"(?<=\s|,|\.)ab\b": "",
    r"(?<=\s|,|\.)ou\b": "",
    r"(?<=\s|,|\.)as\b": "",
    r"(?<=\s|,|\.)gmbh\b": "",
    r"(?<=\s|,|\.)holding\b": "",
    r"(?<=\s|,|\.)holdings\b": "",
    r"(?<=\s|,|\.)corporation\b": "",
    r"(?<=\s|,|\.)industries\b": "",
    r"(?<=\s|,|\.)company limited\b": "",
    r"(?<=\s|,|\.)technology\b": "",
    r"(?<=\s|,|\.)solutions\b": "",
    r"(?<=\s|,|\.)systems\b": "",
    r"(?<=\s|,|\.)services\b": "",
    r"(?<=\s|,|\.)public company\b": "",

    # New patterns to remove dotted abbreviations
    r"\bs\.?p\.?a\.?\b": "",  # Handles "S.p.A." and variations
    r"\bs\.?r\.?l\.?\b": "",  # Handles "S.R.L."
    r"\bs\.?a\.?\b": "",      # Handles "S.A."
}

---

## **Step 3: Funzione di Normalizzazione**

Questa funzione applica una serie di trasformazioni per ottenere nomi uniformi:

- Conversione in minuscolo
- Rimozione di accenti
- Eliminazione di caratteri speciali
- Sostituzione dei suffissi tramite `REPLACEMENTS`

In [12]:
def normalize_name(name):
    """Normalizza i nomi delle aziende."""
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    # 1. Rimuove accenti e caratteri speciali
    name = unidecode.unidecode(name)
    
    # 2. Minuscolo e rimozione spazi extra
    name = name.lower().strip()
    
    # 3. Rimuove testo tra virgolette
    name = re.sub(r'"([^"]+)"','', name) # 3. Rimuove testo tra virgolette
    
    # 4. Applica le sostituzioni dalla mappa REPLACEMENTS
    for pattern, replacement in REPLACEMENTS.items():
        name = re.sub(pattern, replacement, name)
    
    # 5. Pulisce caratteri inutili
    name = re.sub(r"\s*,+\s*", " ", name)
    name = re.sub(r"\s*\.+\s*", " ", name) 
    
    # 6. Sostituisce spazi multipli con uno solo
    name = re.sub(r'\s+', ' ', name)  
    
    # 7. Eliminare punteggiatura indesiderata alla fine 
    name = re.sub(r"[.,\-]+$", "", name)
    
    return name.strip()

---

## **Step 4: Applicazione alla Colonna 'company_name'**

Eseguiamo la normalizzazione sulla colonna contenente i nomi aziendali.

In [13]:
companies_df['normalized_name'] = companies_df['company_name'].apply(normalize_name)
companies_df['modified'] = companies_df['normalized_name'] != companies_df['company_name']

---

## **Step 5: Visualizzazione e salvataggio del Risultato**

Mostriamo i primi risultati per verificare la trasformazione
e salva tutto in un csv

In [14]:
num_modified = companies_df['modified'].sum()
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = companies_df[companies_df['modified']][['company_name', 'normalized_name']]
display(modified_entries.sample(25))

companies_df = companies_df.sort_values(by=['normalized_name'])
companies_df[['company_name', 'normalized_name', 'modified']].to_csv('normalized_names.csv', index=False)

Entries modificate: 37656 su 76808 (49.03%)


,company_name,normalized_name
4427,anritsu corp,anritsu
26031,genertellife spa,genertellife
55819,rugvista group ab,rugvista group
44854,new age professionals ltd,new age professionals
36455,kiewit corp,kiewit
61543,spacerepoãœ(16593350),spacerepoaoe(16593350)
37874,lancomé,lancome
76711,NaN,
2027,air liquide sa,air liquide
57038,sapphire corp ltd,sapphire


---

## **Conclusioni**

Questo approccio garantisce una normalizzazione efficace dei nomi aziendali, riducendo le variazioni dovute a differenze di scrittura, abbreviazioni e formati. È facilmente estendibile aggiornando la mappa `REPLACEMENTS` con nuove regole di sostituzione.



In [15]:
companies_df['company_name'] = companies_df['normalized_name']
companies_df = companies_df.sort_values(by=['company_name'])
companies_df.to_csv('../aziende_normalizzate.csv', index=False)

# **Normalizzazione delle città**

In [16]:
filtered_df = companies_df[companies_df['city'].notna() & (companies_df['city'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
73003,west pharmaceutical,exton
4561,aon insurance reinsurance brokers,milano
75594,zongmu,shanghai
24695,fortra,eden prairie
66579,the sunday times limited,london
60637,softbank robotics uk,london
72598,warner music group,new york
55196,ring,hawthorne
40536,markel international insurance,london
52621,purplle,mumbai


### **Step 1: Eliminare eventuale testo tra parentesi** 
La cosa che sempra più intuitiva da fare è eliminare eventuale testo tra parentesi dopo la città (perdendo un po' di informazione, ma comunque accettabile)

In [17]:
def clean_city(city):
    if pd.isna(city) or not isinstance(city, str):
        return ""
    
    city = re.sub(r"\s*\(.*?\)\s*", "", city).strip()
    return city 

In [18]:
filtered_df['city'] = filtered_df['city'].apply(clean_city)
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
2086,airbus helicopters,grand prairie
11573,bymycar italia,milano
27804,green window,"poltsamaa, poltsamaa vald"
61385,sound technologies,carlsbad
32821,infopro digital usa,new york
51202,princess cruise lines,santa clarita
17151,current lighting,beachwood
32715,ineos automotive limited,london
32714,ineos acetyls uk limited,lyndhurst
19471,ds smith,london


In [19]:
# Apply the changes to the df
companies_df['city'] = companies_df['city'].apply(clean_city)

### **Step 2: Tradurre nomi di città italiane in italiano** 

Bisognerebbe anche indagare se qualche città viene riportata in lingue diverse...

In [20]:
# Query in italiano
milano_df = filtered_df[filtered_df["city"] == "milano"]
roma_df = filtered_df[filtered_df["city"] == "roma"]

# Query in inglese
milan_df = filtered_df[filtered_df["city"] == "milan"]
rome_df = filtered_df[filtered_df["city"] == "rome"]

In [21]:
display(milano_df[['company_name', 'city']])
display(milan_df[['company_name', 'city']])

,company_name,city
562,aberdeen standard investments ireland limited,milano
622,above comparison,milano
623,above comparison,milano
766,accenture,milano
765,accenture,milano
...,...,...
73454,withsecure,milano
74336,yes ticket,milano
74335,yes ticket,milano
75694,zurich italy bank,milano


,company_name,city
57140,satispay,milan
57364,scalapay,milan


In [22]:
display(roma_df[['company_name', 'city']])
display(rome_df[['company_name', 'city']])

,company_name,city
1431,aec underwriting agenzia di assicurazione e ri...,roma
1430,aec underwriting agenzia di assicurazione e ri...,roma
1739,agenzia delle entrate,roma
2463,ald automotive italia,roma
4282,angelini pharma italia aziende chimiche riunit...,roma
...,...,...
68023,trenitalia,roma
69746,universita' degli studi di roma la sapienza,roma
69748,universita' degli studi roma tre,roma
73045,western union payment ireland limited,roma


,company_name,city


Probabilmente, visto la sorgente dati, le aziende italiane sono state estratte da repository italiani o almeno convertite con città in italiano. Solo due aziende (*satispay* e *scalapay*) hanno il nome riportato in inglese.

Le città italiane sono segnate maggiormente in italiano possiamo pensare di **convertire le città con nome inglese in italiano** 

Abbiamo bisogno di un dizionario che mappi i nomi delle città (City-mapping). Evitiamo di scriverlo da zero e ne usiamo uno noto da qualche libreria (*do not reinvent the wheel...*).

Usiamo dunque la libreria `geopy` che con **GeoNames**, un database globale di località, ci permette di convertire i nomi 

In [23]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="city_normalizer")
city_cache = {}

In [24]:
def get_city_in_italian(city):
    if pd.isna(city) or not isinstance(city, str) or city == "":
        return ""

    if city in city_cache:
        print(f"Cache hit! {city}")
        return city_cache[city]
    
    try:
        location = geolocator.geocode(city, exactly_one=True, timeout=10)
        if location and "Italia" in location.address:
            location_it = geolocator.geocode(city, language="it", timeout=10)
            if location_it:
                print(f"translating {location_it}...")
                it_city = location_it.raw.get("display_name", city).split(",")[0].lower().strip()
                city_cache[city] = it_city
                return it_city
    except Exception as e:
        print(f"Errore con {city}: {e}")

    city_cache[city] = city
    return city

In [25]:
def apply_geocoding(city):
    result = get_city_in_italian(city)
    time.sleep(1)
    return result

In [26]:
milan_df['city'] = milan_df['city'].apply(apply_geocoding)
display(milan_df[['company_name', 'city']])

translating Milano, Lombardia, Italia...
Cache hit! milan


C:\Users\emigi\AppData\Local\Temp\ipykernel_14944\1402383521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  milan_df['city'] = milan_df['city'].apply(apply_geocoding)


,company_name,city
57140,satispay,milano
57364,scalapay,milano


**GeoNames** funziona, adesso applichiamolo all'intero dataframe

In [27]:
companies_df['city_translate'] = companies_df.apply(
    lambda row: apply_geocoding(row['city']), 
    axis=1
)
companies_df['modified'] = companies_df['city'] != companies_df['city_translate']

KeyboardInterrupt: 

In [58]:
num_modified = companies_df['modified'].sum()
total_cities = len(companies_df)
percentage_modified = (num_modified / total_cities) * 100

print(f"Città tradotte: {num_modified} su {total_cities} ({percentage_modified:.2f}%)")

translated_cities = companies_df[companies_df['modified']][['company_name', 'city', 'city_translate']]
display(translated_cities.sample(25))

companies_df = companies_df.sort_values(by=['company_name'])
companies_df[['company_name', 'city', 'city_translate']].to_csv('translated_cities.csv', index=False)

Città tradotte: 37656 su 76808 (49.03%)


KeyError: "['city_translate'] not in index"

In [267]:
filtered_df = companies_df[companies_df['country'].notna() & (companies_df['country'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'country']].sample(25))

,company_name,country
73149,wheels india,india
4115,ams,austria
68437,tujia,china
60227,slc agricola,brazil
29792,heidelberger druckmaschinen,germany
23427,finanzchef24,germany
22470,expensify,usa
15530,colt data centre uk limited,united kingdom
55725,rpm international,usa
64695,taisol,taiwan


In [268]:
import re
import pandas as pd

def clean_country(country): 
    if pd.isna(country) or not isinstance(country, str):
        return ""
    
    # Rimuove parentesi tonde ( ), quadre [ ] e singole virgolette '
    country = re.sub(r"[\[\]']", "", country)  
    country = re.sub(r"\s*\(.*?\)\s*", "", country).strip()  
    
    return country


In [269]:
filtered_df['country'] = filtered_df['country'].apply(clean_country)
display(filtered_df[['company_name', 'country']].sample(25))

,company_name,country
18449,dexus,australia
45548,nio,china
3836,american eagle outfitters,usa
45079,newegg,usa
51403,promocollection,australia
14340,chongqing zhifei biological products,china
8147,banque saudi fransi,saudi arabia
72023,vivid seats,usa
46648,o r i martin,italy
15834,compugen,israel


In [270]:
country_mapping = {
    'it': 'italy',            # Codici e abbreviazioni per l'Italia
    'uk': 'united kingdom',   # Abbreviazione per il Regno Unito
    'usa': 'united states',   # Abbreviazione per gli Stati Uniti
    'us': 'united states',    # Abbreviazione per gli Stati Uniti (altro formato)
    'fr': 'france',           # Abbreviazione per la Francia
    'de': 'germany',          # Abbreviazione per la Germania
    'es': 'spain',            # Abbreviazione per la Spagna
    'ca': 'canada',           # Abbreviazione per il Canada
    'au': 'australia',        # Abbreviazione per l'Australia
    'cn': 'china',            # Abbreviazione per la Cina
    'in': 'india',            # Abbreviazione per l'India
    'jp': 'japan',            # Abbreviazione per il Giappone
    'br': 'brazil',           # Abbreviazione per il Brasile
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'mx': 'mexico',           # Abbreviazione per il Messico
    'se': 'sweden',           # Abbreviazione per la Svezia
    'no': 'norway',           # Abbreviazione per la Norvegia
    'fi': 'finland',          # Abbreviazione per la Finlandia
    'nl': 'netherlands',      # Abbreviazione per i Paesi Bassi
    'ch': 'switzerland',      # Abbreviazione per la Svizzera
    'ru': 'russia',           # Abbreviazione per la Russia
    'sa': 'south africa',     # Abbreviazione per il Sudafrica
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'sg': 'singapore'         # Abbreviazione per Singapore
}

In [271]:
import re

# Nuovo dizionario di sostituzione per i codici fiscali italiani
CODE_REPLACEMENTS = {
    r"^it\d{10,16}$": "italy"  # Codici fiscali italiani che iniziano con "it" seguito da 10-16 numeri
}

def normalize_country(country):
    if pd.isna(country) or not isinstance(country, str):
        return ""
    
    country = country.strip().lower()
    
    # Se il valore è un codice fiscale italiano, lo sostituisce direttamente
    for pattern, replacement in CODE_REPLACEMENTS.items():
        if re.match(pattern, country):
            return replacement  
    
    # Se ci sono più paesi separati da virgole, li normalizziamo singolarmente
    country_list = [c.strip() for c in country.split(",")]  # Divide e rimuove spazi
    normalized_list = [country_mapping.get(c, c) for c in country_list]  # Applica il mapping
    
    return ", ".join(normalized_list)  # Ritorna la stringa normalizzata

# Applicare la normalizzazione della colonna 'country'
filtered_df['normalized_country'] = filtered_df['country'].apply(normalize_country)

# Visualizzare una selezione casuale delle righe per verificare
display(filtered_df[['company_name', 'country', 'normalized_country']].sample(25))


,company_name,country,normalized_country
45964,nordic entertainment group (publ),sweden,sweden
59340,sichuan hebang biotechnology limited,china,china
73288,willscot,usa,united states
1538,aeria,japan,japan
47181,one 97 / paytm,india,india
74110,xucker,germany,germany
49672,personalis,usa,united states
9966,bitbank,japan,japan
32882,infratil,new zealand,new zealand
60361,smartswap,estonia,estonia


In [275]:
filtered_df[filtered_df['company_name'] == "international consolidated airlines group"]



,company_id,company_name,trade_name,industry,sector,categories,company_status,company_type,headquarters,address,...,facebook,twitter,pinterest,instagram,investors,region,notes_or_description,normalized_name,modified,normalized_country
33431,faf52c4c18e746ed853f699c2bcb1568,international consolidated airlines group,NaN,NaN,NaN,"['Industries', 'Airlines', 'Aviation', 'Transp...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,international consolidated airlines group,False,"united kingdom, spain"


,company_name,country
236,888,"['Gibralter', 'UK']"
4605,apax global alpha limited,"['UK', 'Guernsey']"
10330,bmo commercial property trust limited,"['UK', 'Guernsey']"
26681,globalworth real estate investments limited,"['UK', 'Guernsey']"
28613,gvc,"['Isle of Man', 'UK']"
29383,harbourvest global private equity limited,"['UK', 'Guernsey']"
30420,hipgnosis songs fund limited,"['UK', 'Guernsey']"
33431,international consolidated airlines group,"['UK', 'Spain']"
33478,international public partnerships ld,"['UK', 'Guernsey']"
35810,kape technologies,"['Isle of Man', 'UK']"
